# Evnironment setup & Raw data collection via web scraping

In [ ]:
!pip install PyPDF2
!pip install git+https://github.com/titipata/scipdf_parser

# !pip install label-studio

import requests
from bs4 import BeautifulSoup
import urllib
import PyPDF2
import os
import tqdm
import spacy
import json
import nltk
nltk.download('punkt')
import nltk.data
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
import re
import certifi
import ssl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 220 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/titipata/scipdf_parser to /tmp/pip-req-build-vlkip68j
  Running command git clone -q https://github.com/titipata/scipdf_parser /tmp/pip-req-build-vlkip68j
     |████████████████████████████████| 105 kB 5.1 MB/s 
     |████████████████████████████████| 2.0 MB 10.9 MB/s 
  Created wheel for scipdf: filename=scipdf-0.1.dev0-py3-none-any.whl size=30167887 sha256=3f94e5c01479ef144a0fab7bda9762707ee48284e496efca3c9daf1529a200db
  Stored in directory: /tmp/pip-ephem-wheel-cache-7i19zzy8/wheels/5f/fa/0a/49da834e340b4295e98cc065f5f455e4fdb183ae9acd749238
Successfully built scipdf


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# NLP conference website link bs request
ssl._create_default_https_context = ssl._create_unverified_context  # deal with ssl certificate verification to avoid SSLCerVerificationError
url = 'https://aclanthology.org/events/conll-2017/'  # data from 2017 -- 2021
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
links = soup.find_all('a')


In [ ]:
# pdf file downloading
file_to_link = {}
i = 0
for link in links:
  if('.pdf' in link.get('href',[])):
    i+=1
    print('downloading file: ', i)
    print(link.get('href',[]))
    content = urllib.request.urlretrieve(link.get('href',[]),'2021 paper'+str(i)+'.pdf')
    print('File ',i,' downloaded')
    file_to_link[i] = link.get('href',[])
print('All pdf files downloaded') 

downloading file:  1
https://aclanthology.org/K17-1.pdf
File  1  downloaded
downloading file:  2
https://aclanthology.org/K17-1000.pdf
File  2  downloaded
downloading file:  3
https://aclanthology.org/K17-1001.pdf
File  3  downloaded
downloading file:  4
https://aclanthology.org/K17-1002.pdf
File  4  downloaded
downloading file:  5
https://aclanthology.org/K17-1003.pdf
File  5  downloaded
downloading file:  6
https://aclanthology.org/K17-1004.pdf
File  6  downloaded
downloading file:  7
https://aclanthology.org/K17-1005.pdf
File  7  downloaded
downloading file:  8
https://aclanthology.org/K17-1006.pdf
File  8  downloaded
downloading file:  9
https://aclanthology.org/K17-1007.pdf
File  9  downloaded
downloading file:  10
https://aclanthology.org/K17-1008.pdf
File  10  downloaded
downloading file:  11
https://aclanthology.org/K17-1009.pdf
File  11  downloaded
downloading file:  12
https://aclanthology.org/K17-1010.pdf
File  12  downloaded
downloading file:  13
https://aclanthology.org/K1

# Extracting sentences from raw pdf data

In [ ]:
# nltk_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# define abbrev in nltk
punkt_param = PunktParameters()
sentence_splitter = PunktSentenceTokenizer(punkt_param)
punkt_param.abbrev_types = set(['i.e','al','e.g','etc','dr', 'vs', 'mr', 'mrs', 'prof'])

spacy_tokenizer = spacy.load("en_core_web_sm")

In [1]:
# extract text from pdf by using PyPDF2
def pdf2sentence(filename):
  def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 60 and y < 800:
        parts.append(text)

  reader = PyPDF2.PdfReader(filename)
  parts = []
  clean_sentence_no_figure = []
  for page in reader.pages:
    page.extract_text(visitor_text=visitor_body)
    text_body = "".join(parts)
  
  # Title detect,
  pos = text_body.find('\n')
  pos_2 = text_body.find('Abstract')

  # print('Position: ',pos)
  # print('Text: ',pos_2)
  text_body = text_body[:pos]+'.' + text_body[pos_2+8:]

  #print(text_body)
  text_body = text_body.replace('-\n','')
  text_body = text_body.replace('\n',' ')

  # filter out references
  ref_index = text_body.rindex("References")
  text_body = text_body[:ref_index]

  all_sentence =  sentence_splitter.tokenize(text_body)
  
  # Drop figure descriptions
  for st in all_sentence: # is figure description or contain only year (2019.)
    if re.search('Figure \d*:',st) or re.match('^\d*.$',st): 
        # print('Dropped:',st)
        continue
    else: clean_sentence_no_figure.append(st)
  print('Sentence num: ',len(clean_sentence_no_figure))


  return clean_sentence_no_figure

In [ ]:
# test data prepare
def test_data_prepare(filename):
  text_body = ''
  with open(filename, 'r',encoding='utf-8') as f:
    for line in f:
      text_body = text_body + line
  
  text_body = text_body.replace('-\n','')
  text_body = text_body.replace('\n',' ')
  all_sentence =  sentence_splitter.tokenize(text_body)
  print(len(all_sentence))
  if(len(all_sentence)>=350):
    return 

  json_object = json.dumps(all_sentence, indent=4)
  with open("test_yubo.json", "w") as outfile:
      outfile.write(json_object)
  return all_sentence


In [ ]:
# extract all sentence in paper contains above words
nlp = spacy.load('en_core_web_sm')
sentences = []
all_sentences=[]

files = os.listdir()
for file in files:
  if 'pdf' in file:
    print(file)
    try:
      all_sentence_per_paper = pdf2sentence(file)
    except ValueError: 
      print('Skip: ',file)
      continue
    for each_sentence in all_sentence_per_paper:
        if len(each_sentence)>=4:
            all_sentences.append(each_sentence)
        if any(ele in each_sentence for ele in MethodName or MetricName or HyperparameterName or TaskName or DatasetName):
            # only tokenize sentence with needed word
            text_tokens = nlp(each_sentence)
            # sentences.append({'text':  ' '.join(tk.text for tk in text_tokens), 'paperID': file})
            sentences.append(each_sentence.strip())

# save sentences with keywords in json format
json_object = json.dumps(sentences, indent=4)
with open("keywords_training.json", "w") as outfile:
    outfile.write(json_object)

# save all sentences(len>=4) in json format
json_object = json.dumps(all_sentences, indent=4)
with open("all_training.json", "w") as outfile:
    outfile.write(json_object)

2018 paper82.pdf
Sentence num:  98
2019 paper129.pdf
Sentence num:  210
2017 paper70.pdf
Sentence num:  188
2018 paper47.pdf
Sentence num:  170
2018 paper73.pdf
Sentence num:  148
2019 paper34.pdf
Sentence num:  189
2021 paper10.pdf
Sentence num:  202
2018 paper45.pdf
Sentence num:  259
2020 paper31.pdf
Sentence num:  177
2019 paper40.pdf
Sentence num:  221
2019 paper90.pdf
Sentence num:  179
2018 paper34.pdf
Sentence num:  198
2018 paper20.pdf
Sentence num:  256
2019 paper68.pdf
Sentence num:  175
2017 paper9.pdf
Sentence num:  250
2019 paper16.pdf
Sentence num:  208
2017 paper31.pdf
Sentence num:  173
2021 paper3.pdf
Sentence num:  187
2019 paper48.pdf
Sentence num:  179
2019 paper83.pdf
Sentence num:  162
2019 paper70.pdf
Sentence num:  205
2017 paper34.pdf
Sentence num:  215
2017 paper22.pdf
Sentence num:  282
2018 paper32.pdf
Sentence num:  192
2018 paper29.pdf
Sentence num:  235
2019 paper139.pdf
Sentence num:  228
2017 paper85.pdf
Sentence num:  167
2018 paper24.pdf
Sentence num

[0, [507], 1, 2, 0, 3, [226], 4, 14, 579, 15, 16, 763, 17, [544], 18, 23, 533, 24, 25, 615, 26, 27, 625, 28, 37, 488, 38, [459], 39, 43, 631, 44, 45, 623, 46, [656], 47, 56, 252, 57, [571], 58, 59, 319, 60, 61, 520, 62, 63, 420, 64, [423, 420, 430, 546, 855], 69, 73, 646, 74, [628], 75, 83, 662, 84, 85, 664, 86, [867], 87, 88, 517, 89, [673], 90, 93, 543, 94, 99, 459, 100, 103, 487, 104, 114, 642, 115, [567], 116, 120, 890, 121, [519], 122, 126, 487, 127, 130, 468, 131, 141, 494, 142, 143, 670, 144, [502], 145, 150, 474, 151, 152, 549, 153, 154, 558, 155, 164, 438, 165, [414], 166, 170, 563, 171, 172, 559, 173, [577], 174, 183, 240, 184, [513], 185, 186, 280, 187, 188, 473, 189, 190, 359, 191, [362, 359, 367, 479, 753], 196, 200, 571, 201, [550], 202, 210, 583, 211, 212, 591, 213, [753], 214, 215, 471, 216, [586], 217, 220, 491, 221, 226, 414, 227, 230, 418, 231, 241, 569, 242, [499], 243, 247, 780, 248, [475], 249, 253, 439, 254, 257, 426, 258, 268, 479, 269, IndirectObject(262, 0, 14

Sentence num:  225
2019 paper17.pdf
Sentence num:  218
2017 paper32.pdf
Sentence num:  220
2017 paper38.pdf
Sentence num:  239
2018 paper16.pdf
Skip:  2018 paper16.pdf
2021 paper17.pdf
Sentence num:  241
2019 paper25.pdf
Sentence num:  229
2020 paper28.pdf
Sentence num:  127
2018 paper103.pdf
Sentence num:  61
2019 paper103.pdf
Sentence num:  164
2017 paper4.pdf
Skip:  2017 paper4.pdf
2021 paper32.pdf
Sentence num:  186
2019 paper97.pdf
Sentence num:  211
2017 paper76.pdf
Sentence num:  175
2020 paper39.pdf
Sentence num:  230
2020 paper51.pdf
Sentence num:  301
2021 paper33.pdf
Sentence num:  211
2019 paper60.pdf
Sentence num:  225
2021 paper53.pdf
Sentence num:  169
2020 paper68.pdf
Sentence num:  161
2019 paper119.pdf
Sentence num:  15
2017 paper69.pdf
Sentence num:  113
2020 paper48.pdf
Sentence num:  222
2020 paper59.pdf
Sentence num:  175
2017 paper42.pdf
Sentence num:  187
2019 paper92.pdf
Sentence num:  203
2017 paper40.pdf
Sentence num:  299
2018 paper12.pdf
Sentence num:  207


# Rule-based word dict 

In [ ]:
#############################################     Metric Name   ######################################################
MetricName = ['recall', 'Recall','precision','Precision', 'F1', 'p-value', 'score','accuracy',
              'AUC','MRR','MAP','RMSE','Area Under the Curve', 'Mean Reciprocal Rank',
              'perplexity','entropy','BPC','bits-per-character','MAE','pearson', 
              'RTE','extract match', 'EM', 'BLEU','ROUGE','SEQVAL','XNLI', 'BERTScore', 'threshold', 'average', 'Average','accuracies','correlation',
              'mean', 'harmonic mean', 'arithmetic mean', 'Correlation','fairness','Fairness','adversarial changes','robustness to noise','logical consistency',
              'Robustness']

#############################################    Method Name    ######################################################

MethodName = ['bi- LSTMs','BERT', 'RoBERTa', 'GPT-3', 'ALBERT', 'XLNet', 'GPT2', 'StructBERT', 
                'T5','ELECTRA','DeBERTa', 'GPT','CodeBERT','BlenderBot','ELMo','ULMFit',
                'PaLM','LSTM','GloVe','Transformer-X','ERNIE','MT-DNN','BigBird','PEGASUS',
               'DAAF','OpenAI','CLIP-T','MIRROR BERT','MNLI','Genre Natural Language Inference','Logistic Regression', 'logistic regression',
              'linear regression','Linear Regression','Decision tree', 'decision tree','SVM', 'Naive Bayes', 'naive bayes',
              'Random Forest', 'random forest', 'PCA', 'Gradient Boosting', 'gradient boosting', 'XGBoost', 'LightGBM','principal component analysis',
              'Principal component analysis','CBOW', 'Word2Vec', 'Skip-Gram', 'Skip Gram', 'nearest neighbors', 'SkipGram',
              'Artificial neural networks ', 'ANNs', 'neural networks','CheckList']

############################################  HyperparameterName  #######################################################

HyperparameterName = ['learning rate', 'number of epochs','epochs','dropout rate','dropout','drop-out','initialization','weight','activation',
                      'hidden layer','hidden unit','batch size', 'optimizer','number of layers',
                      'hidden size','number of self - attention heads','random seeds','momentum',
                      'mini-batch sizes','train-test split ratio','number of activation units in each layer',
                      'number of iterations','number of clusters','filter size','pooling size']

##############################################    Task  Name    #####################################################

TaskName = ['NER','Name entity recognition','automatic summarization' ,'collocation extraction','Information extraction','Information retrieval'
            'entity linking','natural language parsing' ,'part-of-speech tagging','phrase chunking',
            'question answering','relationship extraction','semantic parsing','stemming','shallow parsing',
            'text segmentation','entailment classification task','textual entailment','text simplification','truecasing',
            'terminology extraction','lemmatisation','downstream tasks', 'finetuning','word-sense disambiguation','POS' ,
            'constituent labeling','dependency labeling','named entity labeling','SRL','Semantic role labeling','Semantic Role Labeling','coreference','Coreference',
            'semantic proto-role', 'SPR','SPRL', 'Semantic Proto-role Labeling', 'relation classification', 'downstream','fine tuning'
            'dimensionality reduction','identifying critical failures','sentiment analysis','Question Answering','Natural Language Inference',
            'duplicate question detection','machine comprehension','Document Ranking','Topic modeling','Machine translation',
            'Parts of speech tagging','Word Sense Disambiguation','Grammatical Error Correction','Semantic textual similarity',
            'Question Generation','Image captioning','Fake News Detection','Text generation','emotion analysis','Speech-to-text',
            'Text-to-speech','Dialogue Understanding']


###############################################   Dataset Name   ####################################################

DatasetName = ['CoLA', 'SST-2','MRPC','QQP', 'STS-B','SQuAD','MS MACRO','GLUE','QNLI','WNLI','WordNet','MDT','MDT-NLP','IMDB Reviews',
               'Stanford Sentiment Treebank','Sentiment140','Twitter US Airline Sentiment','20 Newsgroups','Reuters News Dataset',
               'The WikiQA Corpus',"UCI's Spambase",'Yelp Reviews','The Blog Authorship Corpus','2000 HUB5 English','LibriSpeech',
               'Spoken Wikipedia Corpora','Free Spoken Digit Dataset','TIMIT','Enron','Google Books Reviews','Amazon Reviews',
               'Wikipedia Links Data','Blogger Corpus','Gutenberg eBooks List','Jeopardy','Hansards Text Chunks of Canadian Parliament',
               'SMS Spam Collection in English','Recommender Systems Datasets','Project Gutenberg','MultiDomain Sentiment Analysis Dataset',
               'Dictionaries for Movies and Finance','OpinRank Dataset','European Parliament Proceedings Parallel Corpus','Legal Case Reports Dataset',
               'LJ Speech Dataset','M-AI Labs Speech','Noisy Speech Database'] 


In [ ]:
with open('word_dict.txt','w') as f:
  for name in DatasetName:
    f.write(name + ',DatasetName\n')
  for name in TaskName:
    f.write(name + ',TaskName\n')
  for name in MetricName:
    f.write(name + ',MetricName\n')
  for name in MethodName:
    f.write(name + ',MethodName\n')
  for name in HyperparameterName:
    f.write(name + ',HyperparameterName\n')

